In [39]:
import numpy as np

In [40]:
class PlaceholderTensor:
    
    def __init__(self, value):
        self._value = value
        self.subsequent_tensor = None

    def set_value(self, value):
        self._value = value

    def set_subsequent_tensor(self, tensor):
        self.subsequent_tensor = tensor

    def value(self):
        return self._value

In [41]:
class PlusTensor:
    
    def __init__(self, previous, bias):
        self.previous = previous
        self.bias = bias
        self.calculated_value = None
        self.subsequent_tensor = None
        self.previous.set_subsequent_tensor(self)
        self.bias.set_subsequent_tensor(self)

    
    def deriv_error(self, input_tensor):
        return self.subsequent_tensor.deriv_error(self)
        
    def set_subsequent_tensor(self, tensor):
        self.subsequent_tensor = tensor
    
    def value(self):
        if self.calculated_value is None:
            self.calculated_value = self.previous.value() + self.bias.value()
        return self.calculated_value

In [49]:
class MultiplyTensor:
    
    def __init__(self, vect, weight):
        self.vect = vect
        self.weight = weight
        self.subsequent_tensor = None
        self.error_vect = None
        self.error_weight = None
        self.vect.set_subsequent_tensor(self)
        self.weight.set_subsequent_tensor(self)


    
    def deriv_error(self, input_tensor):
        if input_tensor is self.vect:
            return self.deriv_error_vect()
        elif input_tensor is self.weight:
            return self.deriv_error_weight()
        else:
            raise "incorrect tensor given"

    def deriv_error_vect(self):
        if self.error_vect is None:
            self.error_vect = np.dot(self.weight.value(), self.subsequent_tensor.deriv_error(self))
        return self.error_vect
    
    def deriv_error_weight(self):
        if self.error_weight is None:
            self.error_weight = np.outer(self.vect.value(), self.subsequent_tensor.deriv_error(self))
        return self.error_weight
    
    def set_subsequent_tensor(self, tensor):
        self.subsequent_tensor = tensor
    
    def value(self):
        return np.dot(self.vect.value(), self.weight.value())

In [43]:
class SoftMaxCETensor:
    
    def __init__(self, preactivation_tensor, true_y_tensor):
        self.preactivation_tensor = preactivation_tensor
        self.true_y_tensor = true_y_tensor # assume one-hot encoding
        self.calculated_h_values = None
        self.gradient = None
        self.log_loss = None
        self.preactivation_tensor.set_subsequent_tensor(self)
    
    def deriv_error(self, input_tensor):
        if self.gradient is None:
            self.gradient = self.h_value() - self.true_y_tensor.value()
        return self.gradient

    def h_value(self):
        if self.calculated_h_values is None:
            pre_e = np.exp(self.preactivation_tensor.value())
            sums = np.sum(pre_e)
            self.calculated_h_values = pre_e / sums
        return self.calculated_h_values
    
    def value(self):
        if self.log_loss is None:
            y_star_prob = np.sum(np.multiply(self.h_value(), self.true_y_tensor.value()))
            self.log_loss = -np.log(y_star_prob)
        return self.log_loss

In [44]:
class SigmoidTensor:
    
    def __init__(self, pre_activations):
        self.pre_activations = pre_activations
        self.activations = None
        self.gradient = None
        self.subsequent_tensor = None
        
    def deriv_error(self, input_tensor):
        if self.gradient is None:
            self.gradient = np.multiply(self.value(), (1 - self.value()))
        return self.gradient

    def set_subsequent_tensor(self, tensor):
        self.subsequent_tensor = tensor
    
    def value(self):
        if self.activations is None:
            pre_exp = np.exp(self.pre_activations)
            self.activations = pre_exp / (1 + pre_exp)
        return self.activations

In [48]:
x = PlaceholderTensor(np.array([1, 2, 3, 4]))
y = PlaceholderTensor(np.array([0, 1]))

weight = PlaceholderTensor(np.random.uniform(size=(4, 2)))
mult = MultiplyTensor(x, weight)

bias = PlaceholderTensor(np.zeros(shape=(2,)))
plus = PlusTensor(mult, bias)

soft_max = SoftMaxCETensor(plus, y)

print(soft_max.value())
print(soft_max.deriv_error(plus))
print(plus.deriv_error(bias))
print(mult.deriv_error(weight))
print(mult.deriv_error(x))

0.19320874586573267
[ 0.17569012 -0.17569012]
[ 0.17569012 -0.17569012]
[[ 0.17569012 -0.17569012]
 [ 0.35138023 -0.35138023]
 [ 0.52707035 -0.52707035]
 [ 0.70276046 -0.70276046]]
[-0.03524979  0.08523946 -0.07560523 -0.04499989]
